# Parametric Insurance Product Analysis for Soyabean Farmers

This notebook walks through the design, analysis, and pricing of a parametric crop insurance product for Soyabean farmers in Indore Block. The product is designed to protect against heavy rainfall during the harvesting period (1st to 31st October).

## 1. Configuration & Setup

First, we import the necessary libraries and define the core parameters for our insurance product and financial model. These constants are based on the technical challenge specifications.

In [27]:
import pandas as pd
import numpy as np

# --- Configuration & Constants ---
SUM_INSURED_PER_ACRE = 50000
ROLLING_WINDOW_DAYS = 5
DATA_FILE_PATH = 'Historical Weather Data - ERA5-Land Rain Data.csv'

# Trigger definition based on historical percentiles
TRIGGER_PERCENTILES = {
    'Tier 1 (Severe)': 0.80,
    'Tier 2 (Extreme)': 0.90,
    'Tier 3 (Catastrophic)': 0.95
}

# Loadings for calculating the commercial premium
EXPENSE_LOADING = 0.25
RISK_MARGIN = 0.20
TARGET_PREMIUM_RATE = 0.05

## 2. Data Loading and Processing

Here, we load the raw ERA5-Land rainfall data. The function processes the data to calculate a single, area-wide average daily rainfall for the Indore Block by averaging the readings from all 15 grid points for each day.

In [28]:
def load_and_process_data(file_path):
    """
    Loads and processes the raw ERA5-Land rainfall data.
    """
    try:
        df = pd.read_csv(file_path)
        df.columns = df.columns.str.strip().str.replace('\n', '', regex=True)
    except FileNotFoundError:
        print(f"Error: The data file was not found at '{file_path}'")
        return None

    df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
    
    # Calculate the area-wide average rainfall for each day
    daily_avg_rainfall = df.groupby('Date')['ERA5_RAIN (mm)'].mean()

    return daily_avg_rainfall

# Execute the function
daily_rainfall = load_and_process_data(DATA_FILE_PATH)
if daily_rainfall is not None:
    print("Data loaded and processed successfully.")
    print(daily_rainfall.head())

Data loaded and processed successfully.
Date
2005-10-01    0.026840
2005-10-02    0.008593
2005-10-03    0.000900
2005-10-04    0.005847
2005-10-05    0.000000
Name: ERA5_RAIN (mm), dtype: float64


## 3. Index Calculation

The core of the parametric product is the index. We define the index as the **Maximum N-Day Rolling Cumulative Rainfall**. This function calculates this value for each year in the historical dataset.

In [22]:
def calculate_annual_index(daily_data, window_size):
    """
    Calculates the parametric index for each year.
    """
    # Calculate the rolling sum for the entire series
    rolling_sum = daily_data.rolling(window=window_size, min_periods=1).sum()
    
    # Group by year and find the maximum rolling sum for each year
    annual_index_values = rolling_sum.groupby(rolling_sum.index.year).max()
    annual_index_values.name = f'Max {window_size}-Day Rolling Sum (mm)'
    return annual_index_values

# Execute the function
annual_index = calculate_annual_index(daily_rainfall, ROLLING_WINDOW_DAYS)
print("Annual Index Values (2005-2024):")
print(annual_index)

Annual Index Values (2005-2024):
Date
2005     0.604353
2006     7.662733
2007     0.119107
2008    21.294573
2009    96.561073
2010    21.949473
2011     1.097907
2012     2.822193
2013    37.160847
2014     4.059447
2015    11.707520
2016    39.705767
2017    16.734887
2018     0.648807
2019    42.793640
2020    21.469880
2021    48.936333
2022    55.713000
2023     1.511673
2024    49.812940
Name: Max 5-Day Rolling Sum (mm), dtype: float64


## 4. Product Design: Triggers and Payout Calibration

Now we design the product's trigger mechanism. We define three tiers of payouts based on the severity of the rainfall event, determined by percentiles of the historical index distribution. Then, we calibrate the payout percentages for each tier to meet the target commercial premium rate of 5%.

In [23]:
def design_triggers(annual_index, percentiles):
    """
    Defines the trigger thresholds for each payout tier.
    """
    triggers = {
        tier: annual_index.quantile(p) for tier, p in percentiles.items()
    }
    return triggers

def calibrate_payout_slabs(annual_index, triggers, target_rate, expense_load, risk_margin):
    """
    Calibrates the payout percentages for each tier.
    """
    # Determine which years triggered which tier
    tier3_events = (annual_index >= triggers['Tier 3 (Catastrophic)']).sum()
    tier2_events = ((annual_index >= triggers['Tier 2 (Extreme)']) & (annual_index < triggers['Tier 3 (Catastrophic)'])).sum()
    tier1_events = ((annual_index >= triggers['Tier 1 (Severe)']) & (annual_index < triggers['Tier 2 (Extreme)'])).sum()

    total_loading_factor = 1 + expense_load + risk_margin
    required_pure_premium_rate = target_rate / total_loading_factor
    total_payout_percent_over_period = required_pure_premium_rate * len(annual_index)
    
    payout_weights = {'tier1': 1, 'tier2': 2, 'tier3': 4}
    total_weighted_events = (tier1_events * payout_weights['tier1'] + 
                             tier2_events * payout_weights['tier2'] + 
                             tier3_events * payout_weights['tier3'])

    if total_weighted_events == 0:
        return {'Tier 1 (Severe)': 0, 'Tier 2 (Extreme)': 0, 'Tier 3 (Catastrophic)': 0}

    base_payout_unit = total_payout_percent_over_period / total_weighted_events

    calibrated_payouts = {
        'Tier 1 (Severe)': base_payout_unit * payout_weights['tier1'],
        'Tier 2 (Extreme)': base_payout_unit * payout_weights['tier2'],
        'Tier 3 (Catastrophic)': base_payout_unit * payout_weights['tier3']
    }
    return calibrated_payouts

# Execute the functions
triggers = design_triggers(annual_index, TRIGGER_PERCENTILES)
calibrated_payouts = calibrate_payout_slabs(
    annual_index, triggers, TARGET_PREMIUM_RATE, EXPENSE_LOADING, RISK_MARGIN
)

print("--- Product Design ---")
print(f"Index Definition: Maximum {ROLLING_WINDOW_DAYS}-Day Rolling Cumulative Rainfall")
print(f"Sum Insured per Acre: ₹{SUM_INSURED_PER_ACRE:,.0f}")

print("\n[A] Trigger Thresholds (based on historical data):")
for tier, threshold in triggers.items():
    print(f"  - {tier} (>{int(TRIGGER_PERCENTILES[tier]*100)}th percentile): {threshold:.2f} mm")

print("\n[B] Calibrated Payout Slabs (to achieve 5% premium rate):")
for tier, payout_pct in calibrated_payouts.items():
    print(f"  - {tier}: {payout_pct*100:.2f}% of Sum Insured")

--- Product Design ---
Index Definition: Maximum 5-Day Rolling Cumulative Rainfall
Sum Insured per Acre: ₹50,000

[A] Trigger Thresholds (based on historical data):
  - Tier 1 (Severe) (>80th percentile): 44.02 mm
  - Tier 2 (Extreme) (>90th percentile): 50.40 mm
  - Tier 3 (Catastrophic) (>95th percentile): 57.76 mm

[B] Calibrated Payout Slabs (to achieve 5% premium rate):
  - Tier 1 (Severe): 8.62% of Sum Insured
  - Tier 2 (Extreme): 17.24% of Sum Insured
  - Tier 3 (Catastrophic): 34.48% of Sum Insured


## 5. Back-testing and Simulation

To validate the product, we run a historical simulation (back-test) to see how it would have performed from 2005-2024. This shows which years would have triggered a payout and the corresponding amount.

In [24]:
def run_simulation(annual_index, triggers, payouts, sum_insured):
    """
    Runs a historical simulation (back-test) of the product.
    """
    results = []
    for year, index_value in annual_index.items():
        payout_tier = 'None'
        payout_percent = 0.0

        if index_value >= triggers['Tier 3 (Catastrophic)']:
            payout_tier = 'Tier 3 (Catastrophic)'
            payout_percent = payouts['Tier 3 (Catastrophic)']
        elif index_value >= triggers['Tier 2 (Extreme)']:
            payout_tier = 'Tier 2 (Extreme)'
            payout_percent = payouts['Tier 2 (Extreme)']
        elif index_value >= triggers['Tier 1 (Severe)']:
            payout_tier = 'Tier 1 (Severe)'
            payout_percent = payouts['Tier 1 (Severe)']

        payout_amount = payout_percent * sum_insured
        results.append({
            'Year': year,
            f'Max {ROLLING_WINDOW_DAYS}-Day Sum (mm)': index_value,
            'Payout Tier Triggered': payout_tier,
            'Payout (% of SI)': payout_percent * 100,
            'Payout per Acre (₹)': payout_amount
        })

    return pd.DataFrame(results)

# Run the simulation
simulation_results = run_simulation(
    annual_index, triggers, calibrated_payouts, SUM_INSURED_PER_ACRE
)

print("--- Historical Simulation (Back-testing) ---")
simulation_results_display = simulation_results.copy()
simulation_results_display['Payout (% of SI)'] = simulation_results_display['Payout (% of SI)'].map('{:.2f}%'.format)
simulation_results_display['Payout per Acre (₹)'] = simulation_results_display['Payout per Acre (₹)'].map('₹{:,.0f}'.format)
simulation_results_display[f'Max {ROLLING_WINDOW_DAYS}-Day Sum (mm)'] = simulation_results_display[f'Max {ROLLING_WINDOW_DAYS}-Day Sum (mm)'].map('{:.2f}'.format)

print(simulation_results_display.to_string(index=False))

--- Historical Simulation (Back-testing) ---
 Year Max 5-Day Sum (mm) Payout Tier Triggered Payout (% of SI) Payout per Acre (₹)
 2005               0.60                  None            0.00%                  ₹0
 2006               7.66                  None            0.00%                  ₹0
 2007               0.12                  None            0.00%                  ₹0
 2008              21.29                  None            0.00%                  ₹0
 2009              96.56 Tier 3 (Catastrophic)           34.48%             ₹17,241
 2010              21.95                  None            0.00%                  ₹0
 2011               1.10                  None            0.00%                  ₹0
 2012               2.82                  None            0.00%                  ₹0
 2013              37.16                  None            0.00%                  ₹0
 2014               4.06                  None            0.00%                  ₹0
 2015              11.71       

## 6. Final Premium Calculation and Verification

Finally, we verify that our calibrated payouts result in the target 5% commercial premium rate. We calculate the average annual payout from the simulation (the Pure Premium or Burn Cost) and then apply the expense and risk loadings to arrive at the final commercial premium.

In [25]:
print("\n--- Final Premium Calculation Verification ---")
avg_payout_percent = simulation_results['Payout (% of SI)'].mean() / 100
print(f"Average Annual Payout (Pure Premium Rate): {avg_payout_percent:.4f} or {avg_payout_percent:.2%}")

commercial_premium_rate = avg_payout_percent * (1 + EXPENSE_LOADING + RISK_MARGIN)
print(f"Total Loading (Expenses + Risk Margin): {EXPENSE_LOADING + RISK_MARGIN:.0%}")
print(f"Calculated Commercial Premium Rate: {commercial_premium_rate:.4f} or {commercial_premium_rate:.2%}")

final_premium_per_acre = commercial_premium_rate * SUM_INSURED_PER_ACRE
print(f"\nFinal Calibrated Premium per Acre: ₹{final_premium_per_acre:,.2f}")
print("---------------------------------------------")


--- Final Premium Calculation Verification ---
Average Annual Payout (Pure Premium Rate): 0.0345 or 3.45%
Total Loading (Expenses + Risk Margin): 45%
Calculated Commercial Premium Rate: 0.0500 or 5.00%

Final Calibrated Premium per Acre: ₹2,500.00
---------------------------------------------
